In [ ]:
import win32com.client as win32

def excel_to_ppt(excel_path, ppt_path):
    # 1. Lancer Excel
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = False
    wb = excel.Workbooks.Open(excel_path)

    # 3. Sélectionner un graphique
    sheet = wb.Sheets("Feuil1")
    chart = sheet.ChartObjects("Graphique 2")
    chart.Copy()  # copie dans le presse-papier

    # 4. Lancer PowerPoint
    ppt = win32.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    presentation = ppt.Presentations.Open(ppt_path)
    slide = presentation.Slides(1)  # première slide

    # 5. Coller dans le slide
    slide.Shapes.Paste()

    # 6. Sauvegarder et fermer
    presentation.Save()
    wb.Close(SaveChanges=False)
    excel.Quit()

excel_to_ppt("test_sheet.xlsx", "test.pptx")


In [15]:
import os
import win32com.client as win32

def update_ppt_objects_final(excel_path, ppt_path, slide_index, mapping):
    """
    mapping : dict
        clé   = nom de l'objet dans PowerPoint
        valeur = nom du graphique ou de la plage Excel (ChartObject ou Range)
    """

    # Vérification des chemins avant ouverture
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"Fichier Excel introuvable : {excel_path}")
    if not os.path.exists(ppt_path):
        raise FileNotFoundError(f"Fichier PowerPoint introuvable : {ppt_path}")

    # Lancer Excel
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = False
    wb = excel.Workbooks.Open(excel_path)
    sheet = wb.Sheets("Feuil1")

    # Lancer PowerPoint
    ppt = win32.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    presentation = ppt.Presentations.Open(ppt_path)
    slide = presentation.Slides(slide_index)

    for ppt_name, excel_name in mapping.items():
        old_shape = next((s for s in slide.Shapes if s.Name == ppt_name), None)

        if "Graph" in excel_name:  # Graphique Excel
            chart = sheet.ChartObjects(excel_name)
            chart.Copy()
            new_shape = slide.Shapes.Paste()[0]  # Chart copy always returns 1 shape
        else:
            # Plage Excel → coller comme OLE object (vrai Excel éditable)
            rng = sheet.Range(excel_name)
            rng.Copy()

            shape_range = slide.Shapes.PasteSpecial(DataType=8)  # ppPasteOLEObject
            if shape_range is None:
                raise RuntimeError(f"Impossible de coller la plage {excel_name} dans PowerPoint.")
            new_shape = shape_range.Item(1)


        # Conserver position et taille si l'objet existait déjà
        if old_shape:
            new_shape.Left = old_shape.Left
            new_shape.Top = old_shape.Top
            new_shape.Width = old_shape.Width
            new_shape.Height = old_shape.Height
            old_shape.Delete()

        new_shape.Name = ppt_name

    presentation.Save()
    wb.Close(SaveChanges=False)
    excel.Quit()


# Exemple d'utilisation
mapping = {
    "Graphique1": "Graphique1",  # Graphique Excel
    "Tableau1": "A1:D11"          # Plage Excel à garder format
}

update_ppt_objects_final(r"C:\Users\tariq\Desktop\ppt-automate\sheet_test\test_sheet.xlsx", r"C:\Users\tariq\Desktop\ppt-automate\ppt_test\test.pptx", slide_index=1, mapping=mapping)


In [ ]:
import win32com.client as win32

def update_ppt_graph_table_final(excel_path, ppt_path, slide_index, mapping):
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = False
    wb = excel.Workbooks.Open(excel_path)
    
    ppt = win32.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    presentation = ppt.Presentations.Open(ppt_path)
    slide = presentation.Slides(slide_index)

    for ppt_name, excel_name in mapping.items():
        old_shape = next((s for s in slide.Shapes if s.Name == ppt_name), None)

        if "Graph" in excel_name:
            chart = wb.Sheets("Feuil1").ChartObjects(excel_name)
            chart.Copy()
            new_shape = slide.Shapes.Paste()[0]
        else:
            # 🔑 Exécuter la macro Excel pour copier avec format source
            excel.Application.Run("CopyFormattedRange", excel_name)
            shape_range = slide.Shapes.PasteSpecial(DataType=8)  # OLE Object -> conserve format
            new_shape = shape_range.Item(1)

        if old_shape:
            new_shape.Left = old_shape.Left
            new_shape.Top = old_shape.Top
            new_shape.Width = old_shape.Width
            new_shape.Height = old_shape.Height
            old_shape.Delete()

        new_shape.Name = ppt_name

    presentation.Save()
    wb.Close(SaveChanges=False)
    excel.Quit()



# Exemple d'utilisation
mapping = {
    "Graphique1": "Graphique1",  # Graphique Excel
    "Tableau1": "A1:D11"          # Plage Excel à garder format
}

update_ppt_objects_final(r"C:\Users\tariq\Desktop\ppt-automate\sheet_test\test_sheet.xlsm", r"C:\Users\tariq\Desktop\ppt-automate\ppt_test\test.pptx", slide_index=1, mapping=mapping)


In [ ]:
import pandas as pd

df = pd.read_excel(r"sheet_test\test_sheet.xlsx", sheet_name="Feuil1")
df

In [ ]:
import win32com.client as win32

def update_rectangles_from_excel(excel_path, ppt_path, slide_index, mapping):
    """
    mapping : dict
        clé   = nom du rectangle dans PowerPoint
        valeur = adresse de la cellule Excel (ex: "B2")
    """
    # Lancer Excel
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = False
    wb = excel.Workbooks.Open(excel_path)
    sheet = wb.Sheets("Feuil1")

    # Lancer PowerPoint
    ppt = win32.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    presentation = ppt.Presentations.Open(ppt_path)
    slide = presentation.Slides(slide_index)

    for ppt_name, excel_cell in mapping.items():
        # Lire la valeur dans Excel
        value = sheet.Range(excel_cell).Value

        # Trouver la forme dans PPT
        shape = None
        for s in slide.Shapes:
            if s.Name == ppt_name:
                shape = s
                break

        if shape is not None:
            shape.TextFrame.TextRange.Text = str(value)
        else:
            print(f"⚠ Forme '{ppt_name}' introuvable sur la slide {slide_index}")

    # Sauvegarder et fermer
    presentation.Save()
    wb.Close(SaveChanges=False)
    excel.Quit()


# Exemple d'utilisation
mapping = {
    "ZoneTexte1": "C13",
    "ZoneTexte2": "B13"
}

update_rectangles_from_excel("test_sheet.xlsx", "test.pptx", slide_index=1, mapping=mapping)


In [ ]:
import win32com.client as win32
from datetime import datetime

def update_rectangle_from_excel_with_mapping(excel_path, ppt_path, slide_index, mapping):
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = False
    wb = excel.Workbooks.Open(excel_path)
    sheet = wb.Sheets("Feuil1")

    ppt = win32.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    presentation = ppt.Presentations.Open(ppt_path)
    slide = presentation.Slides(slide_index)

    for ppt_name, info in mapping.items():
        cell_address = info["cell"]
        fmt = info.get("format", "default")

        value = sheet.Range(cell_address).Value

        # Appliquer le format choisi
        if fmt == "0decimals":
            formatted_value = f"{value:,.0f}"
        elif fmt == "percent":
            formatted_value = f"{value*100:.0f}%"
        elif fmt == "currency":
            formatted_value = f"${value:,.2f}"
        elif fmt.startswith("date:"):
            date_fmt = fmt.split(":", 1)[1]
            if isinstance(value, datetime):
                formatted_value = value.strftime(date_fmt)
            else:
                formatted_value = str(value)
        else:  # format par défaut
            formatted_value = str(value)

        # Mettre à jour le rectangle
        shape = next((s for s in slide.Shapes if s.Name == ppt_name), None)
        if shape:
            shape.TextFrame.TextRange.Text = formatted_value
        else:
            print(f"⚠ Forme '{ppt_name}' introuvable sur la slide {slide_index}")

    presentation.Save()
    wb.Close(SaveChanges=False)
    excel.Quit()


# Exemple d'utilisation
# mapping = {
#     "Rectangle 1": {"cell": "B2", "format": "2decimals"},
#     "Rectangle 2": {"cell": "C5", "format": "percent"},
#     "Rectangle 3": {"cell": "D7", "format": "currency"},
#     "Rectangle 4": {"cell": "E3", "format": "date:%d/%m/%Y"}
# }

# Exemple d'utilisation
mapping = {
    "ZoneTexte1":  {"cell": "C13", "format": "0decimals"},
    "ZoneTexte2":  {"cell": "B13", "format": "percent"}
}

update_rectangle_from_excel_with_mapping("test_sheet.xlsx", "test.pptx", slide_index=1, mapping=mapping)


### Final functions

In [ ]:
import win32com.client as win32

def update_rectangle_from_excel_with_macro(excel_path, ppt_path, slide_index, mapping):
    # Lancer Excel
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = False
    wb = excel.Workbooks.Open(excel_path)

    # Lancer PowerPoint
    ppt = win32.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    presentation = ppt.Presentations.Open(ppt_path)
    slide = presentation.Slides(slide_index)

    for ppt_name, info in mapping.items():
        cell_address = info["cell"]

        try:
            # 🔑 Exécuter la macro Excel pour copier la cellule avec mise en forme
            excel.Application.Run("CopyFormattedCell", cell_address)
        except Exception as e:
            print(f"❌ Erreur lors de l'exécution de la macro sur {cell_address}: {e}")
            continue

        shape = next((s for s in slide.Shapes if s.Name == ppt_name), None)
        if shape:
            try:
                shape.TextFrame.TextRange.Text = ""  # Vider le texte existant
                shape.TextFrame.TextRange.Paste()  # Coller avec format source
            except Exception as e:
                print(f"⚠ Impossible de coller dans la forme '{ppt_name}': {e}")
        else:
            print(f"⚠ Forme '{ppt_name}' introuvable sur la slide {slide_index}")

    presentation.Save()
    wb.Close(SaveChanges=False)
    excel.Quit()


# ✅ Exemple d'utilisation
mapping = {
    "ZoneTexte1": {"cell": "C13"},
    "ZoneTexte2": {"cell": "B13"}
}

update_rectangle_from_excel_with_macro(
    r"C:\Users\tariq\Desktop\ppt-automate\sheet_test\test_sheet.xlsm",
    r"C:\Users\tariq\Desktop\ppt-automate\ppt_test\test.pptx",
    slide_index=1,
    mapping=mapping
)


In [ ]:
import win32com.client as win32

def update_ppt_graph_table_final(excel_path, ppt_path, slide_index, mapping):
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = False
    wb = excel.Workbooks.Open(excel_path)
    
    ppt = win32.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    presentation = ppt.Presentations.Open(ppt_path)
    slide = presentation.Slides(slide_index)

    for ppt_name, excel_name in mapping.items():
        old_shape = next((s for s in slide.Shapes if s.Name == ppt_name), None)

        if "Graph" in excel_name:
            chart = wb.Sheets("Feuil1").ChartObjects(excel_name)
            chart.Copy()
            new_shape = slide.Shapes.Paste()[0]
        else:
            # 🔑 Exécuter la macro Excel pour copier avec format source
            excel.Application.Run("CopyFormattedRange", excel_name)
            shape_range = slide.Shapes.PasteSpecial(DataType=8)  # OLE Object -> conserve format
            new_shape = shape_range.Item(1)

        if old_shape:
            new_shape.Left = old_shape.Left
            new_shape.Top = old_shape.Top
            new_shape.Width = old_shape.Width
            new_shape.Height = old_shape.Height
            old_shape.Delete()

        new_shape.Name = ppt_name

    presentation.Save()
    wb.Close(SaveChanges=False)
    excel.Quit()



# Exemple d'utilisation
mapping = {
    "Graphique1": "Graphique1",  # Graphique Excel
    "Tableau1": "A1:D11"          # Plage Excel à garder format
}

update_ppt_objects_final(r"C:\Users\tariq\Desktop\ppt-automate\sheet_test\test_sheet.xlsm", r"C:\Users\tariq\Desktop\ppt-automate\ppt_test\test.pptx", slide_index=1, mapping=mapping)
